In [41]:
import os

In [42]:
os.getcwd()

'/opt/ml/PTR/code_script'

In [43]:
os.chdir(os.path.join(os.getcwd(), 'code_script/'))

FileNotFoundError: [Errno 2] No such file or directory: '/opt/ml/PTR/code_script/code_script/'

In [44]:
from arguments import get_args_parser
from templating import get_temps
from modeling import get_model, get_tokenizer
from data_prompt import REPromptDataset
from optimizing import get_optimizer
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import RandomSampler, DataLoader, SequentialSampler
from tqdm import tqdm, trange
import numpy as np
from collections import Counter
import random

In [ ]:
temps

In [5]:
def set_seed(seed):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    if args.n_gpu > 0:
        torch.cuda.manual_seed_all(seed)


args = get_args_parser()

set_seed(args.seed)
tokenizer = get_tokenizer(special=[])
temps = get_temps(tokenizer)

['0', 'per:charges', 'person', 'was', 'charged', 'with', 'event']
{'name': 'per:charges', 'temp': [['the', '<mask>'], ['<mask>', '<mask>', '<mask>'], ['the', '<mask>']], 'labels': [('person',), ('was', 'charged', 'with'), ('event',)]}
['0', 'per:date_of_death', 'person', 'was', 'died', 'on', 'date']
{'name': 'per:date_of_death', 'temp': [['the', '<mask>'], ['<mask>', '<mask>', '<mask>'], ['the', '<mask>']], 'labels': [('person',), ('was', 'died', 'on'), ('date',)]}
['0', 'per:country_of_death', 'person', 'was', 'died', 'in', 'country']
{'name': 'per:country_of_death', 'temp': [['the', '<mask>'], ['<mask>', '<mask>', '<mask>'], ['the', '<mask>']], 'labels': [('person',), ('was', 'died', 'in'), ('country',)]}
['0', 'per:cause_of_death', 'person', 'was', 'died', 'of', 'event']
{'name': 'per:cause_of_death', 'temp': [['the', '<mask>'], ['<mask>', '<mask>', '<mask>'], ['the', '<mask>']], 'labels': [('person',), ('was', 'died', 'of'), ('event',)]}
['0', 'org:founded_by', 'organization', 'was

In [40]:
model.prompt_label_idx

[tensor([  621,  1651, 10014]),
 tensor([  18,  354, 7325]),
 tensor([  334,   919,   920,   962,   998,  1046,  1207,  1270,  1340,  2034,
          2421,  3200,  4095,  4790,  5221,  5407,  8540,  8850, 17117, 21771,
         21821, 25385, 26241, 29853]),
 tensor([ 7,  9, 11, 15, 16, 19, 21, 30, 34]),
 tensor([  194,   247,   343,   346,   515,   621,  1248,  1270,  1651,  6825,
         10014, 46471])]

In [ ]:
klue_re

In [6]:
temps

{'per:charges': {'name': 'per:charges',
  'temp': [['the', '<mask>'],
   ['<mask>', '<mask>', '<mask>'],
   ['the', '<mask>']],
  'labels': [('person',), ('was', 'charged', 'with'), ('event',)]},
 'per:date_of_death': {'name': 'per:date_of_death',
  'temp': [['the', '<mask>'],
   ['<mask>', '<mask>', '<mask>'],
   ['the', '<mask>']],
  'labels': [('person',), ('was', 'died', 'on'), ('date',)]},
 'per:country_of_death': {'name': 'per:country_of_death',
  'temp': [['the', '<mask>'],
   ['<mask>', '<mask>', '<mask>'],
   ['the', '<mask>']],
  'labels': [('person',), ('was', 'died', 'in'), ('country',)]},
 'per:cause_of_death': {'name': 'per:cause_of_death',
  'temp': [['the', '<mask>'],
   ['<mask>', '<mask>', '<mask>'],
   ['the', '<mask>']],
  'labels': [('person',), ('was', 'died', 'of'), ('event',)]},
 'org:founded_by': {'name': 'org:founded_by',
  'temp': [['the', '<mask>'],
   ['<mask>', '<mask>', '<mask>'],
   ['the', '<mask>']],
  'labels': [('organization',), ('was', 'founded', '

- total : {relation_label_id : {token_id : count}, ...}
- 각 [MASK]별  MLM Head가 예측해야할 class label로 사용되는 token id, subj 1개, relation 3개, obj 1개

In [7]:
dataset = REPromptDataset(
    path  = args.data_dir, 
    name = 'val.txt', 
    rel2id = args.data_dir + "/" + "rel2id.json", 
    temps = temps,
    tokenizer = tokenizer)
# dataset.save(path = args.output_dir, name = "val")

{0: {621: 1, 1651: 1, 10014: 1}, 1: {7325: 1, 18: 1, 354: 1}, 2: {1340: 1, 962: 1, 4790: 1, 2421: 1, 5221: 1, 5407: 1, 1270: 1, 25385: 1, 2034: 1, 1207: 1, 919: 1, 8850: 1, 334: 1, 3200: 1, 21771: 1, 17117: 1, 4095: 1, 920: 1, 29853: 1, 8540: 1, 26241: 1, 998: 1, 1046: 1, 21821: 1}, 3: {19: 1, 15: 1, 11: 1, 9: 1, 30: 1, 16: 1, 21: 1, 34: 1, 7: 1}, 4: {515: 1, 1248: 1, 247: 1, 621: 1, 343: 1, 194: 1, 1270: 1, 1651: 1, 6825: 1, 346: 1, 46471: 1, 10014: 1}}
[621, 1651, 10014]
[18, 354, 7325]
[334, 919, 920, 962, 998, 1046, 1207, 1270, 1340, 2034, 2421, 3200, 4095, 4790, 5221, 5407, 8540, 8850, 17117, 21771, 21821, 25385, 26241, 29853]
[7, 9, 11, 15, 16, 19, 21, 30, 34]
[194, 247, 343, 346, 515, 621, 1248, 1270, 1651, 6825, 10014, 46471]


100%|██████████| 22631/22631 [00:19<00:00, 1179.79it/s]


- label_ids : 각 MLM Head에서의 인덱스 번호
- mask_ids : 서브 프롬프트별 토큰 인덱스 인코딩

In [ ]:
toke

In [30]:
dataset.temp_ids

{'per:charges': {'label_ids': [0, 2, 8, 5, 4],
  'mask_ids': [[627, 50264], [50264, 50264, 50264], [627, 50264]]},
 'per:date_of_death': {'label_ids': [0, 2, 3, 3, 6],
  'mask_ids': [[627, 50264], [50264, 50264, 50264], [627, 50264]]},
 'per:country_of_death': {'label_ids': [0, 2, 3, 2, 1],
  'mask_ids': [[627, 50264], [50264, 50264, 50264], [627, 50264]]},
 'per:cause_of_death': {'label_ids': [0, 2, 3, 1, 4],
  'mask_ids': [[627, 50264], [50264, 50264, 50264], [627, 50264]]},
 'org:founded_by': {'label_ids': [1, 2, 13, 7, 5],
  'mask_ids': [[627, 50264], [50264, 50264, 50264], [627, 50264]]},
 'org:founded': {'label_ids': [1, 2, 13, 2, 6],
  'mask_ids': [[627, 50264], [50264, 50264, 50264], [627, 50264]]},
 'per:city_of_death': {'label_ids': [0, 2, 3, 2, 2],
  'mask_ids': [[627, 50264], [50264, 50264, 50264], [627, 50264]]},
 'per:stateorprovince_of_death': {'label_ids': [0, 2, 3, 2, 0],
  'mask_ids': [[627, 50264], [50264, 50264, 50264], [627, 50264]]},
 'per:date_of_birth': {'label_

In [36]:
train_dataset.prompt_label_idx

[tensor([  621,  1651, 10014]),
 tensor([  18,  354, 7325]),
 tensor([  334,   919,   920,   962,   998,  1046,  1207,  1270,  1340,  2034,
          2421,  3200,  4095,  4790,  5221,  5407,  8540,  8850, 17117, 21771,
         21821, 25385, 26241, 29853]),
 tensor([ 7,  9, 11, 15, 16, 19, 21, 30, 34]),
 tensor([  194,   247,   343,   346,   515,   621,  1248,  1270,  1651,  6825,
         10014, 46471])]

In [39]:
train_dataset.prompt_id_2_label[train_dataset[0]['labels']]

tensor([ 1,  2, 13,  2,  6], device='cuda:0')

In [29]:
dataset.prompt_id_2_label

tensor([[ 1,  2, 13,  2,  6],
        [ 1,  0, 16,  6,  8],
        [ 0,  2, 10,  2,  6],
        [ 0,  2,  3,  1,  4],
        [ 0,  0,  5,  6,  3],
        [ 0,  2, 10,  2,  0],
        [ 0,  2,  6,  2,  1],
        [ 0,  2, 10,  2,  1],
        [ 0,  2,  6,  2,  0],
        [ 1,  0,  4,  6, 11],
        [ 0,  2,  6,  2,  2],
        [ 0,  0, 12,  6,  5],
        [ 0,  0, 11,  6,  8],
        [ 2,  1, 20,  0, 10],
        [ 0,  2, 10,  2,  2],
        [ 1,  0, 12,  6,  8],
        [ 1,  2,  1,  1,  9],
        [ 0,  0,  0,  6,  8],
        [ 0,  2,  3,  2,  1],
        [ 0,  0,  2,  6,  5],
        [ 1,  0, 17,  6,  5],
        [ 0,  2,  3,  3,  6],
        [ 1,  0,  1,  6,  8],
        [ 1,  0, 23,  6,  8],
        [ 0,  2,  1,  1,  9],
        [ 1,  2,  1,  1,  8],
        [ 1,  2,  9,  2,  2],
        [ 0,  0, 22,  6,  1],
        [ 1,  2, 14,  7,  5],
        [ 0,  2,  8,  5,  4],
        [ 0,  0,  7,  4,  7],
        [ 1,  0, 17,  8,  3],
        [ 1,  2, 21,  2,  6],
        [ 

In [8]:
val_dataset = REPromptDataset.load(
    path = args.output_dir, 
    name = "val", 
    temps = temps,
    tokenizer = tokenizer,
    rel2id = args.data_dir + "/" + "rel2id.json")

{0: {621: 1, 1651: 1, 10014: 1}, 1: {7325: 1, 18: 1, 354: 1}, 2: {1340: 1, 962: 1, 4790: 1, 2421: 1, 5221: 1, 5407: 1, 1270: 1, 25385: 1, 2034: 1, 1207: 1, 919: 1, 8850: 1, 334: 1, 3200: 1, 21771: 1, 17117: 1, 4095: 1, 920: 1, 29853: 1, 8540: 1, 26241: 1, 998: 1, 1046: 1, 21821: 1}, 3: {19: 1, 15: 1, 11: 1, 9: 1, 30: 1, 16: 1, 21: 1, 34: 1, 7: 1}, 4: {515: 1, 1248: 1, 247: 1, 621: 1, 343: 1, 194: 1, 1270: 1, 1651: 1, 6825: 1, 346: 1, 46471: 1, 10014: 1}}
[621, 1651, 10014]
[18, 354, 7325]
[334, 919, 920, 962, 998, 1046, 1207, 1270, 1340, 2034, 2421, 3200, 4095, 4790, 5221, 5407, 8540, 8850, 17117, 21771, 21821, 25385, 26241, 29853]
[7, 9, 11, 15, 16, 19, 21, 30, 34]
[194, 247, 343, 346, 515, 621, 1248, 1270, 1651, 6825, 10014, 46471]


In [27]:
len(val_dataset.temp_ids), len(val_dataset.set)

(42, 5)

In [9]:
train_batch_size = args.per_gpu_train_batch_size * max(1, args.n_gpu)

In [10]:
train_batch_size

8

In [11]:
val_sampler = SequentialSampler(val_dataset)
val_dataloader = DataLoader(val_dataset, sampler=val_sampler, batch_size=train_batch_size//2)

In [13]:
model = get_model(tokenizer, val_dataset.prompt_label_idx)
optimizer, scheduler, optimizer_new_token, scheduler_new_token = get_optimizer(model, val_dataloader)
criterion = nn.CrossEntropyLoss()

Some weights of the model checkpoint at roberta-large were not used when initializing RobertaModel: ['lm_head.decoder.weight', 'lm_head.layer_norm.weight', 'lm_head.bias', 'lm_head.dense.weight', 'lm_head.layer_norm.bias', 'lm_head.dense.bias']
- This IS expected if you are initializing RobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [14]:
model

Model(
  (model): RobertaModel(
    (embeddings): RobertaEmbeddings(
      (word_embeddings): Embedding(50265, 1024, padding_idx=1)
      (position_embeddings): Embedding(514, 1024, padding_idx=1)
      (token_type_embeddings): Embedding(1, 1024)
      (LayerNorm): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): RobertaEncoder(
      (layer): ModuleList(
        (0): RobertaLayer(
          (attention): RobertaAttention(
            (self): RobertaSelfAttention(
              (query): Linear(in_features=1024, out_features=1024, bias=True)
              (key): Linear(in_features=1024, out_features=1024, bias=True)
              (value): Linear(in_features=1024, out_features=1024, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): RobertaSelfOutput(
              (dense): Linear(in_features=1024, out_features=1024, bias=True)
              (LayerNorm): LayerNorm((

In [23]:
model.model.embeddings

RobertaEmbeddings(
  (word_embeddings): Embedding(50265, 1024, padding_idx=1)
  (position_embeddings): Embedding(514, 1024, padding_idx=1)
  (token_type_embeddings): Embedding(1, 1024)
  (LayerNorm): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
  (dropout): Dropout(p=0.1, inplace=False)
)

In [27]:
train_dataset = REPromptDataset.load(
    path = args.output_dir, 
    name = "train", 
    temps = temps,
    tokenizer = tokenizer,
    rel2id = args.data_dir + "/" + "rel2id.json")

{0: {621: 1, 1651: 1, 10014: 1}, 1: {7325: 1, 18: 1, 354: 1}, 2: {1340: 1, 962: 1, 4790: 1, 2421: 1, 5221: 1, 5407: 1, 1270: 1, 25385: 1, 2034: 1, 1207: 1, 919: 1, 8850: 1, 334: 1, 3200: 1, 21771: 1, 17117: 1, 4095: 1, 920: 1, 29853: 1, 8540: 1, 26241: 1, 998: 1, 1046: 1, 21821: 1}, 3: {19: 1, 15: 1, 11: 1, 9: 1, 30: 1, 16: 1, 21: 1, 34: 1, 7: 1}, 4: {515: 1, 1248: 1, 247: 1, 621: 1, 343: 1, 194: 1, 1270: 1, 1651: 1, 6825: 1, 346: 1, 46471: 1, 10014: 1}}
[621, 1651, 10014]
[18, 354, 7325]
[334, 919, 920, 962, 998, 1046, 1207, 1270, 1340, 2034, 2421, 3200, 4095, 4790, 5221, 5407, 8540, 8850, 17117, 21771, 21821, 25385, 26241, 29853]
[7, 9, 11, 15, 16, 19, 21, 30, 34]
[194, 247, 343, 346, 515, 621, 1248, 1270, 1651, 6825, 10014, 46471]


In [31]:
tokenizer.mask_token_id

50264

In [33]:
tokenizer.decode(train_dataset[0]['input_ids'])

'<s>Zagat Survey, the guide empire that started as a hobby for Tim and Nina Zagat in 1979 as a two-page typed list of New York restaurants compiled from reviews from friends, has been put up for sale, according to people briefed on the decision.the<mask> Zagat<mask><mask><mask>the<mask> 1979</s><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad

In [ ]:
train_dataset.prompt_id_2_label[0, 20]

In [26]:
dataset[2]['mlm_labels']

tensor([-1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1,
        -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1,  1, -1, -1, -1,  1,
         1,  1, -1,  1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1,
        -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1,
        -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1,
        -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1,
        -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1,
        -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1,
        -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1,
        -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1,
        -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1,
        -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1,
        -1, -1, -1, -1, -1, -1, -1, -1, 

In [35]:
dataset[4]['mlm_labels']

tensor([-1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1,
        -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1,
        -1, -1,  1, -1, -1, -1,  1,  1,  1, -1,  1, -1, -1, -1, -1, -1, -1, -1,
        -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1,
        -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1,
        -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1,
        -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1,
        -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1,
        -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1,
        -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1,
        -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1,
        -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1,
        -1, -1, -1, -1, -1, -1, -1, -1, 

In [15]:
val_dataset.prompt_id_2_label[[0, 3, 2]

tensor([[ 1,  2, 13,  2,  6],
        [ 0,  2,  3,  1,  4],
        [ 0,  2, 10,  2,  6]], device='cuda:0')